In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score, accuracy_score

# 결측치 처리
from sklearn.impute import SimpleImputer

In [24]:
df1 = pd.read_csv("실습 파일/01_Data.csv")
df1.head(3)

,Index,Member_ID,Sales_Type,Contract_Type,Channel,Datetime,Term,Payment_Type,Product_Type,Amount_Month,Customer_Type,Age,Address1,Address2,State,Overdue_count,Overdue_Type,Gender,Credit_Rank,Bank
0,1,66758234,렌탈,일반계약,영업방판,2022-05-05,60,CMS,DES-1,96900,개인,42.0,경기도,경기도,계약확정,0,없음,여자,9.0,새마을금고
1,2,66755948,렌탈,교체계약,영업방판,2023-02-19,60,카드이체,DES-1,102900,개인,39.0,경기도,경기도,계약확정,0,없음,남자,2.0,현대카드
2,3,66756657,렌탈,일반계약,홈쇼핑/방송,2022-02-27,60,CMS,DES-1,96900,개인,48.0,경기도,경기도,계약확정,0,없음,여자,8.0,우리은행


In [25]:
df1['Credit_Rank']

0        9.0
1        2.0
2        8.0
3        5.0
4        8.0
        ... 
51296    NaN
51297    8.0
51298    1.0
51299    2.0
51300    8.0
Name: Credit_Rank, Length: 51301, dtype: float64

### 결측치 처리

In [26]:
# 결측치 처리(숫자, 문자 가능)

# 어떤 값이 들어가도, 결측치는 평균값으로 채워줌
SimpleImputer(strategy='mean').fit_transform(df1[['Credit_Rank']])

array([[9.],
       [2.],
       [8.],
       ...,
       [1.],
       [2.],
       [8.]])

In [27]:
# 어떤 값이 들어가도, 결측치는 특정값으로 채워줌
SimpleImputer(strategy='constant',fill_value=df1['Credit_Rank'].mean()).fit_transform(df1[['Credit_Rank']])

array([[9.],
       [2.],
       [8.],
       ...,
       [1.],
       [2.],
       [8.]])

In [28]:
# 어떤 값이 들어가도, 결측치는 최빈값으로 채워줌
SimpleImputer(strategy='most_frequent').fit_transform(df1[['Bank']])

array([['새마을금고'],
       ['현대카드'],
       ['우리은행'],
       ...,
       ['롯데카드'],
       ['롯데카드'],
       ['신한은행']], dtype=object)

### 스케일링
- Standard Scaler
    - 평균 0 , 표준편차 1 
    - 이상치에 예민함
- Robust Scaler
    - 이상치가 많을 때 사용함
    - 값에 중앙값을 빼고, IQR(25%~75%)로 나눔 
    - 이상치에 아주 강함
- MinMax Scaler
    - 최댓값 1, 최소값 0 을 변경해줌
    - 값에 음수가 있다면 -1~1 사이로 해줌
    - 이상치에 Standard 보다 더 예민함
    - 개인적으로 비추이지만, 데이터에 따라 다름
    - 이미지데이터에 자주 사용함 (픽셀 값들이니깐, 이상치가 존재할 수가 없어서)
    - 예를 들어, 삼성전자가 3만 ~ 8만 사이, 삼성SDI(50만 ~ 80만) 사이에서 움직인다고 할 때, 서로 겹치는 경우가 없을 때 사용해줌

In [29]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

sample = df1[['Amount_Month', 'Age','Term']]
sample
# 이 데이터의 경우 Amount_Month에 쏠릴 확률이 매우 큼
# 그래서 표준화가 필요!

,Amount_Month,Age,Term
0,96900,42.0,60
1,102900,39.0,60
2,96900,48.0,60
3,66900,39.0,12
4,66900,60.0,12
...,...,...,...
51296,96900,47.0,60
51297,96900,42.0,60
51298,120900,65.0,39
51299,96900,54.0,60


In [32]:
df2 = df1.dropna()

sample = df2[['Amount_Month', 'Age','Term']].to_numpy()
target = df2['Bank'].to_numpy()

mean = sample.mean(axis=0)
std = sample.std(axis=0)

(sample-mean)/std

array([[ 0.16584319, -0.73154113,  0.29120713],
       [ 0.63822156, -1.00387913,  0.29120713],
       [ 0.16584319, -0.18686513,  0.29120713],
       ...,
       [ 2.05535665,  1.35638355, -1.57237835],
       [ 0.16584319,  0.35781088,  0.29120713],
       [ 0.16584319,  0.26703154,  0.29120713]])

In [33]:
ss = StandardScaler()
ss.fit_transform(sample)

array([[ 0.16584319, -0.73154113,  0.29120713],
       [ 0.63822156, -1.00387913,  0.29120713],
       [ 0.16584319, -0.18686513,  0.29120713],
       ...,
       [ 2.05535665,  1.35638355, -1.57237835],
       [ 0.16584319,  0.35781088,  0.29120713],
       [ 0.16584319,  0.26703154,  0.29120713]])

In [34]:
train_x, test_x, train_y, test_y = train_test_split(sample, target)

In [35]:
ss = StandardScaler()

# 기존의 방법으로 진행했을 때는, test_scaled를 할 때, train_x의 평균과 표준편차를 사용했음
# 하지만, 스케일링 방법을 사용한다면
# fit_transform에 train_x 를 학습시키고
# test_x의 경우에는 fit 할 필요없이 transform 만 진행한다면 자동적으로 됨!
train_scaled = ss.fit_transform(train_x)
test_scaled = ss.transform(test_x)


# 이렇게 진행할 수 있음
ss.fit(train_x)
train_scaled = ss.transform(train_x)
test_scaled = ss.transform(test_x)

### Label Encoder
레이블 인코더를 하는 이유는 텍스트마이닝할 때 문제가 생겨서 임!           
컴퓨터는 단어를 이해하지 못해서...the, is, ..., the 0, school : 53000

In [38]:
# 이렇게 안해도 된당! 라이브러리 있슈
dic = {}
box = []
for i in df1['Channel']:
    if i not in dic:
        dic[i] = len(dic)
    box.append(dic[i])
dic, box

({'영업방판': 0,
  '홈쇼핑/방송': 1,
  '재계약': 2,
  '전문매장H': 3,
  '전문매장Z': 4,
  '대형마트H': 5,
  '대형마트E': 6,
  '홈쇼핑/인터넷': 7,
  '전단홍보': 8,
  '직영유통사': 9,
  '대형마트A': 10,
  '대형마트N': 11,
  '대형마트C': 12,
  '직영계열사B': 13,
  '자체홈페이지': 14,
  '직영계열사A': 15},
 [0,
  0,
  1,
  2,
  2,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  2,
  1,
  3,
  2,
  0,
  2,
  4,
  0,
  0,
  4,
  4,
  2,
  1,
  2,
  0,
  2,
  2,
  0,
  5,
  1,
  6,
  0,
  2,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  2,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  3,
  0,
  0,
  5,
  2,
  6,
  1,
  6,
  6,
  0,
  1,
  1,
  1,
  0,
  7,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  4,
  1,
  0,
  0,
  0,
  5,
  1,
  0,
  0,
  0,
  3,
  2,
  1,
  1,
  3,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  3,
  3,
  2,
  1,
  1,
  0,
  0,
  0,
  4,
  0,
  0,
  0,
  3,
  0,
  3,
  2,
  4,
  2,
  0,
  0,
  2,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  

In [39]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit_transform(df1['Channel'])

array([ 5,  5, 14, ...,  5, 14, 14])

In [40]:
encoder.classes_
# 0 은 대형마트A, 5는 영업방판

array(['대형마트A', '대형마트C', '대형마트E', '대형마트H', '대형마트N', '영업방판', '자체홈페이지',
       '재계약', '전단홍보', '전문매장H', '전문매장Z', '직영계열사A', '직영계열사B', '직영유통사',
       '홈쇼핑/방송', '홈쇼핑/인터넷'], dtype=object)

In [42]:
# Label Encoder를 사용하다보면 큰 숫자에 쏠릴 확률이 존재하기 때문에
# 한 열에 1이 하나 있는 pd.get_dummies를 더 추천함!

# 보기 편하라고 astype(int)를 한 것이지, 할 필요는 없슈
pd.get_dummies(df1['Channel']).astype(int)

# 그리고 get_dummies는 One Hot Encoder로 바로 할 수 있어여
# 해당되는 값만 1로 두고 나머지를 0으로 하는 방법

,대형마트A,대형마트C,대형마트E,대형마트H,대형마트N,영업방판,자체홈페이지,재계약,전단홍보,전문매장H,전문매장Z,직영계열사A,직영계열사B,직영유통사,홈쇼핑/방송,홈쇼핑/인터넷
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51296,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51297,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51298,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
51299,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [43]:
# 편하게 구성
def func(x):
    if x =='계약확정':
        return "정상"
    else:
        return "해약"

df1['Target'] = df1['State'].apply(func)
df1['Target']

0        정상
1        정상
2        정상
3        정상
4        해약
         ..
51296    정상
51297    정상
51298    정상
51299    정상
51300    정상
Name: Target, Length: 51301, dtype: object

In [57]:
X = df1[['Product_Type', "Amount_Month", "Age", "Gender", "Credit_Rank"]]
y = df1['Target'].

train_x, test_x, train_y, test_y = train_test_split(X,y,test_size = 0.2)

### 파이프라인 만들기

이 땐 numpy 안해줘도 됨!      
numpy로 변경하게 된다면 값들이 어떤 데이터인지 구분할 수 없게 됨     
그래서 데이터프레임 형태로 컬럼명을 알아야 진행할 수 있기 때문에    
target의 경우에는 numpy해도 상관은 없지만, 그냥 뭐 같이 안하기.
+ 예시           
pipe = make_column_transformer((numeric_pipe, numeric_list),(category_pipe, category_list))

In [75]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

In [59]:
# 숫자형 데이터의 컬럼만 뽑는 쉬운 방법!!!
numeric_list = X.describe().columns
numeric_list

Index(['Amount_Month', 'Age', 'Credit_Rank'], dtype='object')

In [60]:
# include = object로 범주형 데이터의 컬럼만 불러올 수 있음!
category_list = X.describe(include='object').columns
category_list

Index(['Product_Type', 'Gender'], dtype='object')

In [61]:
# 숫자 데이터를 넣어주면 일단 누락값을 평균값으로 처리
# 그리고 표준화할 거임!!
numeric_pipe = make_pipeline(SimpleImputer(strategy = 'mean'), StandardScaler())
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [67]:
# 범주형 데이터를 넣어주면 일단 누락값을 최빈값으로 처리
# 그리고 표준화할 거임!!
category_pipe = make_pipeline(SimpleImputer(strategy = 'most_frequent'), OneHotEncoder())
category_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder())])

In [68]:
pipe = make_column_transformer((numeric_pipe, numeric_list),(category_pipe, category_list))
pipe

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['Amount_Month', 'Age', 'Credit_Rank'], dtype='object')),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 Index(['Product_Type', 'Gender'], dtype='object'))])

In [69]:
model_pipe = make_pipeline(pipe, DecisionTreeClassifier())
model_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['Amount_Month', 'Age', 'Credit_Rank'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  Index(['Product_Type', 'Gender'], dtype='object'))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [70]:
model_pipe.fit(train_x, train_y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['Amount_Month', 'Age', 'Credit_Rank'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  Index(['Product_Type', 'Gender'], dtype='object'))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [71]:
model_pipe.predict(test_x)

array(['정상', '정상', '정상', ..., '정상', '정상', '정상'], dtype=object)

In [76]:
report = classification_report(test_y, model_pipe.predict(test_x))
print(report)

              precision    recall  f1-score   support

          정상       0.99      1.00      0.99     10130
          해약       0.12      0.02      0.04       131

    accuracy                           0.99     10261
   macro avg       0.55      0.51      0.52     10261
weighted avg       0.98      0.99      0.98     10261



#### 파이프라인 만들기 실습
데이터 불러오기, 문제집 정답지, 분할, 표준화, 모델, 학습, 평가

In [101]:
df1 = pd.read_csv("실습 파일/pima_indians.csv")
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Class                     768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [96]:
df1['Class'].value_counts()

Class
0    500
1    268
Name: count, dtype: int64

In [133]:
X = df1.loc[:, ~(df1.columns == "Class")]
X = df1.iloc[:, :-1]
y = df1['Class']

train_x, test_x, train_y, test_y = train_test_split(X,y,test_size = 0.2)

새로운 파이프라인 구축 방법

In [131]:
model_pipe = make_pipeline(StandardScaler(), DecisionTreeClassifier())
model_pipe.fit(train_x, train_y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [144]:
pred = model_pipe.predict(test_x)

report = classification_report(pred, test_y)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.96      0.95        99
           1       0.93      0.91      0.92        55

    accuracy                           0.94       154
   macro avg       0.94      0.93      0.94       154
weighted avg       0.94      0.94      0.94       154



학습한 내용 복습

In [114]:
inpute_pipe = make_pipeline(SimpleImputer(strategy='mean'))

In [115]:
model_pipe = make_pipeline(inpute_pipe, DecisionTreeClassifier())
model_pipe

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('simpleimputer', SimpleImputer())])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [116]:
model_pipe.fit(train_x, train_y)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('simpleimputer', SimpleImputer())])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [117]:
model_pipe.predict(test_x)

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0],
      dtype=int64)

In [118]:
report = classification_report(test_y, model_pipe.predict(test_x))
print(report)

              precision    recall  f1-score   support

           0       0.81      0.69      0.75       110
           1       0.43      0.59      0.50        44

    accuracy                           0.66       154
   macro avg       0.62      0.64      0.62       154
weighted avg       0.70      0.66      0.68       154



### 성능향상 방법
+ 특성공학          
파생변수를 늘려나가는 것,,, 변수의 값을 제곱하거나, 두개의 값을 곱하거나 등등
    - poly는 성능이 안 좋을 때 사용해볼 값, 추천은 아님!
+ 교차검증



### 과적합인지 확인하는 방법 : 훈련데이터로 평가
+ 과대적합
    훈련 데이터 정확도 >>>>>> 시험데이터 정확도              
    두 데이터 그룹의 값이 큰 차이가 나타나면 과대적합난것임
    - 잘 훈련됐다 : 2~4%
    - 과대적합 의심 : 8~10%

+ 과소적합
    훈련 데이터 정확도 < 시험데이터 정확도 이거나 둘다 낮게 나오는 경우

In [134]:
# pima 데이터
X = df1.iloc[:, :-1]
y = df1['Class'].to_numpy()

train_x, test_x, train_y, test_y = train_test_split(X,y,test_size = 0.2)

dt = DecisionTreeClassifier()
dt.fit(train_x, train_y)

# 과적합인지 확인하는 방법 : 훈련데이터로 평가
print(dt.score(test_x, test_y))
print(dt.score(train_x, train_y)) 

0.6233766233766234
1.0


#### 특성공학 

In [142]:
from sklearn.preprocessing import PolynomialFeatures

# 기존 (768,8) 늘리면 (768,45)
poly = PolynomialFeatures()

X_poly = poly.fit_transform(X)
print(X.shape)
print(X_poly.shape)

train_x, test_x, train_y, test_y = train_test_split(X,y,test_size = 0.2)

dt = DecisionTreeClassifier()
dt.fit(train_x, train_y)

# 과적합인지 확인하는 방법 : 훈련데이터로 평가
print(dt.score(test_x, test_y))
print(dt.score(train_x, train_y)) 

(768, 8)
(768, 45)
0.6688311688311688
1.0


#### 교차검증

In [143]:
from sklearn.model_selection import cross_val_score
df = pd.read_csv("실습 파일/pima_indians.csv")

X = df1.iloc[:, :-1].to_numpy()
y = df1['Class'].to_numpy()

dt = DecisionTreeClassifier()

# cv로 5가지로 나누어서 평가하는 것
# 마지막에 평균을 내서 확인하는거
scores = cross_val_score(dt, X,y,cv=5)
scores.mean()

0.7071810542398778

#### K 최근접 단점
- 가벼운 데이터에 적합
- 모든 위치의 값을 저장하고 외워야해서
- 그래서 무거운 데이터를 하면 매우 느려짐

#### 트리기반 단점
- 학습범위를 벗어나면 예측력이 떨어짐

